In [2]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.layers import TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, RepeatVector
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import sys
import heapq
import seaborn as sns
from pylab import rcParams

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 12, 5

In [14]:
path = 'data.txt'
text = open(path)
text = str(text.read())
textList = exec(text)
ListOfWords

['Abdominal pain',
 'Abdominal redness',
 'Abdominal swelling',
 'Abnormal sweating',
 'Acne',
 'Allergy',
 'Anal Fissure',
 'Anal pain',
 'Anemia',
 'Anhedonia',
 'Ankle pain',
 'Anxiety',
 'Appendicitis',
 'Arm pain',
 'Arm swelling',
 'Arm weakness',
 'Armpit pain',
 'Armpit swelling',
 'Attention deficit',
 'Back pain',
 'Bad breath',
 'Bad or bitter taste',
 'Black or tarry stool',
 'Bladder Infection (UTI)',
 'Blood in stool',
 'Blood in urine',
 'Blurry vision',
 'Bronchitis',
 'Bulging eye',
 'Burning or painful urination',
 'Buttocks pain',
 'Calf pain',
 'Calf swelling',
 'Cheek pain',
 'Cheek swelling',
 'Chest pain',
 'Chills',
 'Confusion',
 'Constipation',
 'Cool bluish skin',
 'Cough',
 'COVID-19',
 'Decreased appetite',
 'Decreased hearing',
 'Decreased urination',
 'Decreased vision',
 'Dental pain',
 'Diarrhea',
 'Difficulty urinating',
 'Dizziness',
 'Drooping eyelid',
 'Dry mouth',
 'Dry mucous membranes',
 'Dry skin',
 'Ear discharge',
 'Ear pain',
 'Ear pressure',

In [15]:
print('corpus length:', len(ListOfWords))

corpus length: 113


In [16]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(f'unique chars: {len(chars)}')

unique chars: 53


In [17]:
SEQUENCE_LENGTH = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i: i + SEQUENCE_LENGTH])
    next_chars.append(text[i + SEQUENCE_LENGTH])
print(f'num training examples: {len(sentences)}')

num training examples: 679


In [18]:
X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

C:\Users\hp\AppData\Local\Temp\ipykernel_10452\293202360.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(sentences), SEQUENCE_LENGTH, len(chars)), dtype=np.bool)
C:\Users\hp\AppData\Local\Temp\ipykernel_10452\293202360.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [19]:
sentences[100]

"g',\n 'Attention deficit',\n 'Back pain',\n"

In [20]:
next_chars[100]

' '

In [21]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQUENCE_LENGTH, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [22]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, y, validation_split=0.05, batch_size=128, epochs=20, shuffle=True).history

C:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/20
6/6 [==============================] - 5s 218ms/step - loss: 3.8867 - accuracy: 0.0884 - val_loss: 4.0011 - val_accuracy: 0.1176
Epoch 2/20
6/6 [==============================] - 1s 86ms/step - loss: 3.3285 - accuracy: 0.1287 - val_loss: 3.3844 - val_accuracy: 0.0588
Epoch 3/20
6/6 [==============================] - 1s 89ms/step - loss: 3.1042 - accuracy: 0.1938 - val_loss: 3.3597 - val_accuracy: 0.1176
Epoch 4/20
6/6 [==============================] - 1s 109ms/step - loss: 3.0387 - accuracy: 0.1860 - val_loss: 3.1234 - val_accuracy: 0.1765
Epoch 5/20
6/6 [==============================] - 1s 109ms/step - loss: 2.8708 - accuracy: 0.2667 - val_loss: 3.3912 - val_accuracy: 0.2353
Epoch 6/20
6/6 [==============================] - 1s 98ms/step - loss: 2.7254 - accuracy: 0.2930 - val_loss: 2.9183 - val_accuracy: 0.2647
Epoch 7/20
6/6 [==============================] - 1s 88ms/step - loss: 2.4554 - accuracy: 0.3752 - val_loss: 2.8740 - val_accuracy: 0.2647
Epoch 8/20
6/6 [========

In [23]:
model.save('keras_model.h5')
pickle.dump(history, open("history.p", "wb"))

In [24]:
model = load_model('keras_model.h5')
history = pickle.load(open("history.p", "rb"))

In [29]:
import pandas as pd
import numpy as np
import textdistance
import re
from collections import Counter
words = []
with open('data.txt', 'r') as f:
    file_name_data = f.read()
file_name_data = exec(file_name_data)
V = set(ListOfWords)
words = ListOfWords.copy()
print(f"Top ten words in the text are:{words[0:10]}")
print(f"Total Unique words are {len(V)}.")

Top ten words in the text are:['Abdominal pain', 'Abdominal redness', 'Abdominal swelling', 'Abnormal sweating', 'Acne', 'Allergy', 'Anal Fissure', 'Anal pain', 'Anemia', 'Anhedonia']
Total Unique words are 113.


In [30]:
print(ListOfWords)

['Abdominal pain', 'Abdominal redness', 'Abdominal swelling', 'Abnormal sweating', 'Acne', 'Allergy', 'Anal Fissure', 'Anal pain', 'Anemia', 'Anhedonia', 'Ankle pain', 'Anxiety', 'Appendicitis', 'Arm pain', 'Arm swelling', 'Arm weakness', 'Armpit pain', 'Armpit swelling', 'Attention deficit', 'Back pain', 'Bad breath', 'Bad or bitter taste', 'Black or tarry stool', 'Bladder Infection (UTI)', 'Blood in stool', 'Blood in urine', 'Blurry vision', 'Bronchitis', 'Bulging eye', 'Burning or painful urination', 'Buttocks pain', 'Calf pain', 'Calf swelling', 'Cheek pain', 'Cheek swelling', 'Chest pain', 'Chills', 'Confusion', 'Constipation', 'Cool bluish skin', 'Cough', 'COVID-19', 'Decreased appetite', 'Decreased hearing', 'Decreased urination', 'Decreased vision', 'Dental pain', 'Diarrhea', 'Difficulty urinating', 'Dizziness', 'Drooping eyelid', 'Dry mouth', 'Dry mucous membranes', 'Dry skin', 'Ear discharge', 'Ear pain', 'Ear pressure', 'Ear swelling', 'Elbow pain', 'Emotional stress', 'Erec

In [31]:
word_freq = {}  
word_freq = Counter(words)
print(word_freq.most_common()[0:10])

[('Abdominal pain', 1), ('Abdominal redness', 1), ('Abdominal swelling', 1), ('Abnormal sweating', 1), ('Acne', 1), ('Allergy', 1), ('Anal Fissure', 1), ('Anal pain', 1), ('Anemia', 1), ('Anhedonia', 1)]


In [32]:
probs = {}     
Total = sum(word_freq.values())    
for k in word_freq.keys():
    probs[k] = word_freq[k]/Total

In [35]:
def my_autocorrect(input_word):
    input_word = input_word.lower()
    if input_word in V:
        return('Your word seems to be correct')
    else:
        sim = [1-(textdistance.Jaccard(qval=2).distance(v,input_word)) for v in word_freq.keys()]
        df = pd.DataFrame.from_dict(probs, orient='index').reset_index()
        df = df.rename(columns={'index':'Word', 0:'Prob'})
        df['Similarity'] = sim
        output = df.sort_values(['Similarity', 'Prob'], ascending=False).head()
        return(output)

In [36]:
my_autocorrect('haerlosad')

,Word,Prob,Similarity
106,Hair loss,0.00885,0.142857
66,Eye floaters,0.00885,0.117647
83,Fever,0.00885,0.090909
21,Bad or bitter taste,0.00885,0.083333
103,Goiter,0.00885,0.083333
